source: [Wassertein ACGAN with Keras](https://myurasov.github.io/2017/09/24/wasserstein-gan-keras.html)

In [1]:
import os
os.environ['KERAS_BACKEND'] = 'tensorflow'

In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import keras
import keras.backend as K
from keras.datasets import mnist
from keras import layers, models, optimizers, callbacks, initializers
from keras.utils.generic_utils import Progbar

Using TensorFlow backend.


In [3]:
print("Keras:", keras.__version__)


Keras: 2.0.8


In [4]:
# random seed
RND = 777

# output settings
RUN = 'F'
OUT_DIR = 'out/' + RUN
TENSORBOARD_DIR = 'model/' + RUN
SAVE_SAMPLE_IMAGES = False

BATCH_SIZE = 100
ITERATIONS = 20000

# size of the random vector used to initialize G
Z_SIZE = 100

# number of iterations D is trained for per each G iteration
D_ITERS = 5


In [5]:
np.random.seed(RND)

In [6]:
if not os.path.isdir(OUT_DIR): os.makedirs(OUT_DIR)

### Loss function for Discriminator

In [7]:
# basically return mean(y_pred)
# but with ability to inverse it for minimization (when y_true == -1)
def wasserstein(y_true, y_pred):
    return K.mean(y_true * y_pred)

### Creating Discriminator

In [8]:
def create_D():
    
    # weights are initialized from normal distribution with below params
    weight_init = initializers.RandomNormal(mean=0., stddev=0.02)
    
    input_image = layers.Input(shape=(28, 28, 1), name='input_image')
    
    x = layers.Conv2D(32,
                      (3, 3),
                      padding='same',
                      name='conv_1',
                      kernel_initializer=weight_init)(input_image)
    x = layers.LeakyReLU()(x)
    x = layers.MaxPool2D(pool_size=2)(x)
    x = layers.Dropout(0.3)(x)
    
    x = layers.Conv2D(64,
                      (3, 3),
                      padding='same',
                      name='conv_2',
                      kernel_initializer=weight_init)(x)
    x = layers.MaxPool2D(pool_size=1)(x)
    x = layers.LeakyReLU()(x)
    x = layers.Dropout(0.3)(x)
    
    x = layers.Conv2D(128,
                      (3, 3), 
                      padding='same',
                      name='conv_3',
                      kernel_initializer=weight_init)(x)
    x = layers.MaxPool2D(pool_size=2)(x)
    x = layers.LeakyReLU()(x)
    x = layers.Dropout(0.3)(x)
    
    x = layers.Conv2D(256,
                      (3, 3),
                      padding='same',
                      name='conv_4',
                      kernel_initializer=weight_init)(x)
    x = layers.MaxPool2D(pool_size=1)(x)
    x = layers.LeakyReLU()(x)
    x = layers.Dropout(0.3)(x)
    
    features = layers.Flatten()(x)
    
    output_is_fake = layers.Dense(1,
                                  activation='linear',
                                  name='output_is_fake')(features)
    output_class = layers.Dense(10,
                                activation='softmax',
                                name='output_class')(features)
    return models.Model(inputs=[input_image],
                        outputs=[output_is_fake, output_class], name='D')
    

In [9]:
def create_G(Z_SIZE=Z_SIZE):
    DICT_LEN = 10
    EMBEDDING_LEN = Z_SIZE
    
    weight_init = initializers.RandomNormal(mean=0., stddev=0.02)
    
    input_class = layers.Input(shape=(1, ), dtype='int32', name='input_class')
    
    e = layers.Embedding(DICT_LEN,
                         EMBEDDING_LEN,
                         embeddings_initializer='glorot_uniform')(input_class)
    embedded_class = layers.Flatten(name='embedded_class')(e)
    
    input_z = layers.Input(shape=(Z_SIZE, ), name='input_z')
    
    # hadamard product
    h = layers.multiply([input_z, embedded_class], name='h')
    
    # cnn part
    x = layers.Dense(1024)(h)
    x = layers.LeakyReLU()(x)
    
    x = layers.Dense(128 * 7 * 7)(x)
    x = layers.LeakyReLU()(x)
    x = layers.Reshape((7, 7, 128))(x)
    
    x = layers.UpSampling2D(size=(2, 2))(x)
    x = layers.Conv2D(256, (5, 5), padding='same', kernel_initializer=weight_init)(x)
    x = layers.LeakyReLU()(x)
    
    x = layers.UpSampling2D(size=(2, 2))(x)
    x = layers.Conv2D(128, (5, 5), padding='same', kernel_initializer=weight_init)(x)
    x = layers.LeakyReLU()(x)
    
    x = layers.Conv2D(1,
                      (2, 2),
                      padding='same',
                      activation='tanh', 
                      name='output_generated_image',
                      kernel_initializer=weight_init)(x)
    return models.Model(inputs=[input_z, input_class], outputs=x, name='G')

In [16]:
D = create_D()

In [23]:
G = create_G()